In [1]:
import pandas as pd 
pd.set_option('max_colwidth',200)

In [2]:
data = pd.read_csv(r"F:\temp\190306\part11.csv")
JD_stations_ZJ = pd.read_excel(r"G:\work\logistica\stations\JDStations\stations\gaode\JDstations_zhejiang_gaode.xlsx")

d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,12,26,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns

Index(['订单编号', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '订单来源', '店铺', 'AB类型',
       '订单金额', '订单生成时间', '支付完成时间', '交易完成时间', '发货时间', '入网用户姓名', '入网手机号',
       '入网身份证号', '联系人电话号码', '收货人姓名', '收货地址', '收货人电话号码', '所在省`/`市`/`县',
       '入网人所在省/市/县', '号码归属地', '配送方式', 'CPS推荐人', '订单备注', '物流单号', '物流签收时间',
       '承运商', '写卡渠道', '销售品编号', '销售品名称', '销售品类型', '套餐', '主号码', '副号码', '靓号等级',
       '靓号低消', '靓号预存款', '营业厅送货方式', '是否线下模式', '是否线下转线上', '线下转线上原因',
       '营业厅送货iccid', '发货量', '签收量', '激活量', '月拦截量', '所属省', '所属市', '产品分类', '激活时效',
       '性别', '签收时效', '交付时效', '年龄', '配送时效', 'ORDER_ID', 'MAIL_NO',
       'LOGISTICS_COMPANY', '收货省市县', '写卡渠道号', '索引', '流转信息', '最后送达时间', '网点'],
      dtype='object')

In [4]:
df = data[['订单编号', '收货地址','所在省`/`市`/`县', '物流单号','物流签收时间','承运商','LOGISTICS_COMPANY','最后送达时间','网点']]
# df = data.copy()

In [5]:
split = pd.DataFrame((x.split('/') for x in df['所在省`/`市`/`县']), index=df.index, columns=['所在省','所在市','所在县'])
df = pd.merge(df,split,left_index=True,right_index=True)

In [6]:
df["所在省"].value_counts()

四川    18972
浙江     5986
Name: 所在省, dtype: int64

In [7]:
df[['快递员','最后一站','快递员电话']] = df['网点'].str.extract('配送员(\\w+)，您的订单已到达(\\w+)，.*，均可给我打电话(\\d+)', expand=False)
df['导航地址'] = df['所在省'] + df['所在市'] + df['收货地址']

In [8]:
dict_carrier ={'ems':'EMS','邮政':'EMS','100000137':'圆通','ZHAIJISONG':'宅急送','jingdongwuliu':'京东物流','yunda':'韵达','shunfeng':'顺丰','EMS物流':'EMS'}
df =df.replace(dict_carrier)
df["承运商"].value_counts()

京东物流    24958
Name: 承运商, dtype: int64

In [9]:
dict_province = {'焦作市':'河南','河南省':'河南','河北省':'河北','贵州省':'贵州','湖南省':'湖南','广东省':'广东','江苏省':'江苏','山西省':'山西',
        '吉林省':'吉林','山东省':'山东','广西壮族自治区':'广西','湖北省':'湖北','陕西省':'陕西','浙江省':'浙江','福建省':'福建',
        '内蒙古自治区':'内蒙古','云南省':'云南','辽宁省':'辽宁','江西省':'江西','四川省':'四川','黑龙江省':'黑龙江','安徽省':'安徽',
        '甘肃省':'甘肃','西藏自治区':'西藏','青海省':'青海','宁夏回族自治区':'宁夏',
        '海南省':'海南','六安市':'安徽','东莞市':'广东','新疆维吾尔自治区':'新疆'}
df = df.replace(dict_province)
df["所在省"].value_counts()

四川    18972
浙江     5986
Name: 所在省, dtype: int64

In [10]:
df_JD = df[df["承运商"] == "京东物流"]

In [11]:
df_ZJ_JD = df_JD[df_JD["所在省"] == "浙江"]
print("浙江省样本数：{}".format(df_ZJ_JD.shape[0]))

浙江省样本数：5986


In [12]:
JD_stations_ZJ.columns

Index(['cityId', 'countyId', 'provinceId', 'siteAddress', 'siteBusinessName',
       'siteName', 'siteTelephone', 'cityname', 'siteAddress_all',
       'siteAdcode_gaode', 'siteCity_gaode', 'siteCountry_gaode',
       'siteDistrict_gaode', 'siteFormatted_address_gaode', 'siteLevel_gaode',
       'siteLocation_gaode', 'siteNumber_gaode', 'siteProvince_gaode',
       'siteStreet_gaode', 'siteTownship_gaode', 'site_Lng_gaode',
       'site_Lat_gaode'],
      dtype='object')

In [13]:
df_ZJ_JD.columns

Index(['订单编号', '收货地址', '所在省`/`市`/`县', '物流单号', '物流签收时间', '承运商',
       'LOGISTICS_COMPANY', '最后送达时间', '网点', '所在省', '所在市', '所在县', '快递员', '最后一站',
       '快递员电话', '导航地址'],
      dtype='object')

In [14]:
JD_ZJ = pd.merge(df_ZJ_JD,JD_stations_ZJ,how="left",left_on="最后一站",right_on="siteName")

In [15]:
# print("df:{}".format(df.shape[0]))
print("JD_stations_ZJ:{}".format(JD_stations_ZJ.shape[0]))
print("JD_ZJ:{}".format(JD_ZJ.shape[0]))

JD_stations_ZJ:447
JD_ZJ:5986


In [16]:
JD_ZJ.columns

Index(['订单编号', '收货地址', '所在省`/`市`/`县', '物流单号', '物流签收时间', '承运商',
       'LOGISTICS_COMPANY', '最后送达时间', '网点', '所在省', '所在市', '所在县', '快递员', '最后一站',
       '快递员电话', '导航地址', 'cityId', 'countyId', 'provinceId', 'siteAddress',
       'siteBusinessName', 'siteName', 'siteTelephone', 'cityname',
       'siteAddress_all', 'siteAdcode_gaode', 'siteCity_gaode',
       'siteCountry_gaode', 'siteDistrict_gaode',
       'siteFormatted_address_gaode', 'siteLevel_gaode', 'siteLocation_gaode',
       'siteNumber_gaode', 'siteProvince_gaode', 'siteStreet_gaode',
       'siteTownship_gaode', 'site_Lng_gaode', 'site_Lat_gaode'],
      dtype='object')

In [17]:
JD_ZJ["siteName"].isnull().value_counts()

False    5250
True      736
Name: siteName, dtype: int64

In [18]:
JD_ZJ = JD_ZJ.drop(columns=['承运商', '所在省`/`市`/`县','网点'])

In [19]:
JD_ZJ_list = JD_ZJ[JD_ZJ["siteName"].isnull() == False]
JD_ZJ_NA = JD_ZJ[JD_ZJ["siteName"].isnull() == True]

In [20]:
print("JD_ZJ_list:{}".format(JD_ZJ_list.shape[0]))
print("JD_ZJ_NA:{}".format(JD_ZJ_NA.shape[0]))

JD_ZJ_list:5250
JD_ZJ_NA:736


In [21]:
JD_ZJ_NA["最后一站"].isnull().value_counts()
# JD_ZJ_NA["siteName"].isnull().value_counts()


False    687
True      49
Name: 最后一站, dtype: int64

In [22]:
JD_ZJ_NA = JD_ZJ_NA.dropna(subset=['最后一站'])

In [23]:
JD_ZJ_jingdongpai = JD_ZJ_NA[JD_ZJ_NA["最后一站"].str.contains('京东派|众包站')]
JD_ZJ_otherNA = JD_ZJ_NA[JD_ZJ_NA["最后一站"].str.contains("京东派|众包站") == False]

In [24]:
# JD_ZJ_jingdongpai['最后一站'].value_counts()
JD_ZJ_otherNA['最后一站'].value_counts()

杭州晟虞营业部    24
杭州黄龙营业部    14
温州雪山营业部     8
温州绣山营业部     2
扬州汊河营业部     1
邯郸丛台营业部     1
长沙青园营业部     1
吕梁孝义营业部     1
五常营业部       1
庆阳西峰营业部     1
娄底娄星营业部     1
南昌象湖营业部     1
长沙中信营业部     1
兰州西湖营业部     1
娄底南苑营业部     1
西安泾渭营业部     1
泉州清源营业部     1
Name: 最后一站, dtype: int64

In [26]:
len(JD_ZJ_otherNA['最后一站'])

61

In [25]:
JD_ZJ_list.to_excel(r"G:\work\logistica\info\shenglist\JD_ZJ_list.xlsx",index = False)
JD_ZJ_NA.to_excel(r"G:\work\logistica\info\shenglist\JD_ZJ_NA.xlsx",index = False)
JD_ZJ_list.to_csv(r"G:\work\logistica\info\shenglist\JD_ZJ_list.csv",index = False)